In [ ]:
!pip install yt-dlp
!pip install faster-whisper
!pip install fpdf
!pip install requests
!pip install pdfplumber python-docx
!pip install pyngrok aiohttp nest_asyncio flask flask-cors
!apt-get install fonts-dejavu-core ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=41a51d421d931aa6022e1baa53afeae099fbe05447054538dcad51165e54ef29
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.

In [ ]:
import os
import json
import requests
import textwrap
import glob
import time
import asyncio
import aiohttp
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import concurrent.futures

import yt_dlp  # using yt-dlp for audio download
from faster_whisper import WhisperModel
from fpdf import FPDF

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

import nest_asyncio
nest_asyncio.apply()

# ===============================
# Configuration
# ===============================
API_KEY = "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"  # Replace with your actual key
CHUNK_SIZE = 10000  # Increase chunk size to 10,000 characters

# ===============================
# Global Prompt for Summarization
# ===============================
SYSTEM_PROMPT = (
    "You are an AI assistant that summarizes and structures information from multiple sources "
    "(YouTube videos, textbooks, and documents) into an easy-to-read format.\n\n"
    "Given an input (a transcript, textbook chapter, or document), generate a structured summary that "
    "scales with the length of the input. The format should be:\n\n"
    "1. **Title & Metadata**\n"
    "   - Source Type: (Video, Book, or Document)\n"
    "   - Title: [Extracted from input]\n"
    "   - Author/Speaker: [If available]\n"
    "   - Date: [If available]\n"
    "   - Duration/Page Count: [Estimated from input]\n\n"
    "2. **Quick Summary** (Short and concise for small inputs, more detailed for long inputs).\n\n"
    "3. **Structured Breakdown:**\n"
    "   - For videos: Timeline-based summary with timestamps.\n"
    "   - For books/documents: Section-based summary with chapter titles and key points.\n\n"
    "4. **Key Concepts & Definitions** (Table format).\n\n"
    "5. **Self-Check Questions** (3-5 questions, scaling with content depth).\n\n"
    "6. **Actionable Insights** (What to do & common mistakes to avoid).\n\n"
    "7. **Further Learning** (Related videos, books, and resources).\n\n"
    "Adjust the depth of explanation, number of bullet points, and breakdown sections proportionally "
    "to the length of the input. Ensure clarity, conciseness, and logical organization."
)

# ===============================
# Helper Functions for Different Input Types
# ===============================
def download_audio(youtube_url):
    """
    Downloads the best audio from a YouTube URL using yt-dlp.
    Cleans the URL and uses a cookies file (cookies.txt) if available.
    Returns the downloaded file path.
    """
    parsed = urlparse(youtube_url)
    query_params = parse_qs(parsed.query)
    if 'v' in query_params:
        clean_query = {'v': query_params['v'][0]}
        parsed = parsed._replace(query=urlencode(clean_query))
        youtube_url = urlunparse(parsed)
        print("Cleaned URL:", youtube_url)

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'audio.%(ext)s',
        'quiet': False
    }
    # Use cookies if a cookies.txt file is present
    cookies_file = "cookies.txt"
    if os.path.exists(cookies_file):
        ydl_opts["cookiefile"] = cookies_file
        print("Using cookies file:", cookies_file)

    print("Downloading audio from:", youtube_url)
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([youtube_url])
        except Exception as e:
            print("yt-dlp encountered an error:", e)

    files = glob.glob("audio.*")
    if files:
        return files[0]
    else:
        raise FileNotFoundError("No audio file found after download.")

def transcribe_audio(audio_path):
    """
    Transcribes audio using faster-whisper on GPU.
    """
    model = WhisperModel("base", device="cuda")
    segments, _ = model.transcribe(audio_path)
    transcript = "\n".join([segment.text for segment in segments])
    return transcript

def extract_text_from_pdf(pdf_path):
    import pdfplumber
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n"
    return all_text

def extract_text_from_docx(docx_path):
    import docx
    doc = docx.Document(docx_path)
    full_text = [para.text for para in doc.paragraphs]
    return "\n".join(full_text)

def chunk_text(text, max_chars=CHUNK_SIZE):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end
    return chunks

# ===============================
# Asynchronous Summarization Functions
# ===============================
async def async_summarize_text(session, text):
    payload = {
        "model": "deepseek/deepseek-r1-distill-llama-70b:free",
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text}
        ],
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://www.kaggle.com",
        "X-Title": "Multi-Source Summarizer"
    }
    async with session.post("https://openrouter.ai/api/v1/chat/completions", json=payload, headers=headers) as resp:
        resp_json = await resp.json()
        print("Async LLM API Response:", resp_json)
        return resp_json.get("choices", [{}])[0].get("message", {}).get("content", "")

async def parallel_summarize(chunks):
    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.create_task(async_summarize_text(session, chunk)) for chunk in chunks]
        results = await asyncio.gather(*tasks)
    return results

# ===============================
# Synchronous Summarization Function
# ===============================
def summarize_text_sync(text):
    payload = {
        "model": "deepseek/deepseek-r1-distill-llama-70b:free",
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text}
        ],
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://www.kaggle.com",
        "X-Title": "Multi-Source Summarizer"
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, data=json.dumps(payload))
    print("LLM API Response:", response.json())
    return response.json().get("choices", [{}])[0].get("message", {}).get("content", "")

def save_as_pdf(text, filename="YouTube_Notes.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    try:
        pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
        pdf.set_font('DejaVu', '', 12)
    except RuntimeError:
        pdf.set_font("Arial", size=12)
    for line in text.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(filename, "F")
    print(f"✅ PDF saved as {filename}")
    return filename

def save_as_markdown(text, filename="YouTube_Notes.md"):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"✅ Markdown file saved as {filename}")
    return filename

# ===============================
# Main Modular Pipeline Function
# ===============================
def modular_pipeline(input_source, input_type):
    timings = {}
    start_total = time.time()

    if input_type.lower() == "youtube":
        start_download = time.time()
        audio_path = download_audio(input_source)
        end_download = time.time()
        timings["download_audio"] = end_download - start_download

        start_transcribe = time.time()
        extracted_text = transcribe_audio(audio_path)
        end_transcribe = time.time()
        timings["transcription"] = end_transcribe - start_transcribe

    elif input_type.lower() == "pdf":
        start_pdf = time.time()
        extracted_text = extract_text_from_pdf(input_source)
        end_pdf = time.time()
        timings["pdf_extraction"] = end_pdf - start_pdf

    elif input_type.lower() == "docx":
        start_docx = time.time()
        extracted_text = extract_text_from_docx(input_source)
        end_docx = time.time()
        timings["docx_extraction"] = end_docx - start_docx

    elif input_type.lower() == "text":
        if os.path.exists(input_source):
            with open(input_source, "r", encoding="utf-8") as f:
                extracted_text = f.read()
        else:
            extracted_text = input_source
        timings["text_extraction"] = 0
    else:
        raise ValueError("Unsupported input type. Choose from 'youtube', 'pdf', 'docx', 'text'.")

    # Use asynchronous summarization for long inputs
    if len(extracted_text) > CHUNK_SIZE:
        chunks = chunk_text(extracted_text, max_chars=CHUNK_SIZE)
        start_chunks = time.time()
        chunk_summaries = asyncio.run(parallel_summarize(chunks))
        end_chunks = time.time()
        timings["chunked_summarization"] = end_chunks - start_chunks

        combined_summary = "\n\n".join(chunk_summaries)
        print("Combined Chunk Summary:\n", combined_summary)  # Debug print

        start_final = time.time()
        final_summary = summarize_text_sync(combined_summary)
        end_final = time.time()
        timings["final_summarization"] = end_final - start_final

        # Fallback: if final_summary is empty, use combined_summary directly
        if not final_summary.strip():
            print("Final summary empty, using combined summary as fallback.")
            final_summary = combined_summary
            timings["final_summarization"] = 0
    else:
        start_sum = time.time()
        final_summary = summarize_text_sync(extracted_text)
        end_sum = time.time()
        timings["summarization"] = end_sum - start_sum
        chunk_summaries = [final_summary]

    total_time = time.time() - start_total
    timings["total_time"] = total_time

    pdf_file = save_as_pdf(final_summary)
    md_file = save_as_markdown(final_summary)

    return {
        "final_summary": final_summary,
        "chunk_summaries": chunk_summaries,
        "extracted_text_length": len(extracted_text),
        "timings": timings,
        "pdf_file": pdf_file,
        "markdown_file": md_file
    }

# ===============================
# Flask App Integration
# ===============================
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

@app.route("/summarize", methods=["POST"])
def summarize_endpoint():
    # Check if the request is multipart/form-data (i.e. file upload)
    if request.content_type.startswith("multipart/form-data"):
        file = request.files.get("file")
        input_type = request.form.get("input_type") or "pdf"  # Default to pdf if not provided
        if not file:
            return jsonify({"error": "No file provided"}), 400
        # Save the uploaded file
        filename = file.filename
        file.save(filename)
        input_source = filename
    else:
        data = request.get_json()
        print("Received JSON:", data)
        if not data:
            return jsonify({"error": "No JSON data provided"}), 400
        input_type = data.get("input_type")
        input_source = data.get("input_source")
        if not input_type or not input_source:
            return jsonify({"error": "Missing required parameters: input_type and input_source"}), 400

    try:
        result = modular_pipeline(input_source, input_type)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ===============================
# Run Flask with ngrok
# ===============================
public_url = ngrok.connect(5000)
print("Public URL:", public_url)
app.run(port=5000)


Public URL: NgrokTunnel: "https://be5c-34-118-242-81.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 18:25:31] "OPTIONS /summarize HTTP/1.1" 200 -


Received JSON: {'input_type': 'text', 'input_source': 'Gayan was not an ordinary pig. While most pigs were content to roll in the mud and munch on apples, Gayan had dreams—big dreams. He wanted to see the world beyond Farmer Ted’s barnyard. His pink snout quivered with excitement every time a bird flew overhead, every time the wind carried the scent of adventure from faraway places.  One evening, as the sun dipped below the horizon, painting the sky in hues of orange and purple, Gayan made up his mind. He would escape and explore the world! But he needed a plan. Unlike the barn cats who could slip through the smallest gaps, or the clever hens who had their secret tunnels, Gayan had to rely on his wits.  The opportunity came sooner than expected. One morning, Farmer Ted left the gate slightly ajar while carrying a heavy sack of feed. Gayan’s heart raced. This was it! With a deep breath, he charged through the opening and dashed toward freedom, his little hooves kicking up dust.  Gayan’s

INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 18:25:44] "POST /summarize HTTP/1.1" 200 -


LLM API Response: {'id': 'gen-1741371931-6QXVdgVew5SzhBefyKe3', 'provider': 'Chutes', 'model': 'deepseek/deepseek-r1-distill-llama-70b', 'object': 'chat.completion', 'created': 1741371931, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'Alright, I need to create a structured summary based on the given fairy tale. Let\'s see, the story is about a pig named Gayan who wants to explore the world. So, first, I\'ll gather metadata. The source is likely a document or storybook chapter. The title I can infer as "The Adventures of Gayan the Pig." No author mentioned, but it\'s a fairy tale. Duration or page count I can estimate as short.\n\nNext, the quick summary should capture the main journey: Gayan leaves the farm, meets an owl, crosses a river with beavers\' help, visits a town, and decides to go home.\n\nFor the structured breakdown, since it\'s a document, I\'ll section it into introduction, 

In [ ]:
!ngrok authtoken 2txFnbNlfj624fYptdwRI0gxHZP_6Y3qYrsVtpxZWHH74uHAZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
